In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 705.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 143.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 142.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [ ]:
data = pd.read_csv("/content/ner_dataset.csv",encoding="latin",on_bad_lines='skip')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
def get_dict(data,token_or_tag):
  tok2idx ,idx2tok = {},{}
  if token_or_tag == 'token':
    vocab = list(set(data['Word'].to_list()))
  else:
    vocab = list(set(data['Tag'].to_list()))
  idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
  tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
  return tok2idx, idx2tok

token2idx,idx2token = get_dict(data,'token')
tag2idx,idx2tag = get_dict(data,'tag')

In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.fillna(method='ffill', axis=0)

data_group = data_fillna.groupby(['Sentence #'],as_index=False)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))
data.head()

/tmp/ipython-input-3061450886.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_fillna = data.fillna(method='ffill', axis=0)


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,4412,2
1,NaN,of,IN,O,17195,2
2,NaN,demonstrators,NNS,O,17312,2
3,NaN,have,VBP,O,19277,2
4,NaN,marched,VBN,O,5234,2


In [ ]:
def get_pad_train_test_val(data_group, data):

    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [ ]:
seed(2)
tensorflow.random.set_seed(2)

In [ ]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [ ]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [ ]:
def get_bilstm_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
model_bilstm_lstm.build((None, input_length))
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

You must install pydot (`pip install pydot`) for `plot_model` to work.
26/26 ━━━━━━━━━━━━━━━━━━━━ 24s 658ms/step - accuracy: 0.8293 - loss: 1.6391 - val_accuracy: 0.9681 - val_loss: 0.4473
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 577ms/step - accuracy: 0.9676 - loss: 0.4214 - val_accuracy: 0.9681 - val_loss: 0.3540
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 570ms/step - accuracy: 0.9677 - loss: 0.3577 - val_accuracy: 0.9681 - val_loss: 0.2927
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 562ms/step - accuracy: 0.9677 - loss: 0.3301 - val_accuracy: 0.9681 - val_loss: 0.2808
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 552ms/step - accuracy: 0.9677 - loss: 0.3024 - val_accuracy: 0.9681 - val_loss: 0.2180
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 553ms/step - accuracy: 0.9677 - loss: 0.2536 - val_accuracy: 0.9682 - val_loss: 0.2021
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 545ms/step - accuracy: 0.9678 - loss: 0.2282 - val_accuracy: 0.9683 - val_loss: 0.1670
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 543ms/step - accuracy: 0.9679 - loss: 0.2100 - val_accuracy: 0.9684 - va

In [ ]:
while True:
  nlp = spacy.load('en_core_web_sm')
  text = nlp(input("Enter text: "))
  displacy.render(text, style = 'ent', jupyter=True)

Enter text: hey i am sarvesh


Enter text: i am Loop


Enter text: my name is Loop


Enter text: i am working at Google


Enter text: i am the ceo of LOOP-info.tech


KeyboardInterrupt: Interrupted by user

In [ ]:
model_bilstm_lstm.save('ner_model.h5')

import pickle
with open('token2idx.pkl', 'wb') as f:
    pickle.dump(token2idx, f)

with open('tag2idx.pkl', 'wb') as f:
    pickle.dump(tag2idx, f)

with open('maxlen.pkl', 'wb') as f:
    pickle.dump(input_length, f)